In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
# Check for GPU
import tensorflow as tf
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

if IN_COLAB:
    print("We're running Colab")
else:
    print(tf.config.list_physical_devices())
    print('\nCUDA GPU: ' + str(tf.test.is_gpu_available(cuda_only=True)))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

CUDA GPU: False


# Data Preprocessing

In [3]:
import os
os.chdir('..')
df = pd.read_csv('./daily01-ithaca/daily01-NY_Ithaca_13_E.csv', header = 0, index_col = 0)

In [4]:
df

,WBANNO,LST_DATE,CRX_VN,LONGITUDE,LATITUDE,T_DAILY_MAX,T_DAILY_MIN,T_DAILY_MEAN,T_DAILY_AVG,P_DAILY_CALC,...,SOIL_MOISTURE_5_DAILY,SOIL_MOISTURE_10_DAILY,SOIL_MOISTURE_20_DAILY,SOIL_MOISTURE_50_DAILY,SOIL_MOISTURE_100_DAILY,SOIL_TEMP_5_DAILY,SOIL_TEMP_10_DAILY,SOIL_TEMP_20_DAILY,SOIL_TEMP_50_DAILY,SOIL_TEMP_100_DAILY
0,64758,20041027,1.201,-76.25,42.44,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64758,20041028,1.201,-76.25,42.44,12.7,-0.3,6.2,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64758,20041029,1.201,-76.25,42.44,16.3,2.5,9.4,9.7,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64758,20041030,1.201,-76.25,42.44,17.5,10.5,14.0,14.5,1.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,64758,20041031,1.201,-76.25,42.44,17.0,9.1,13.1,12.6,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,64758,20231101,2.622,-76.25,42.44,3.5,-2.2,0.6,0.2,3.0,...,0.352,0.328,0.329,0.364,0.021,7.0,7.7,8.7,10.0,11.6
305,64758,20231102,2.622,-76.25,42.44,6.1,-3.3,1.4,1.0,0.0,...,0.352,0.323,0.325,0.357,0.021,6.4,6.9,7.8,9.3,11.3
306,64758,20231103,2.622,-76.25,42.44,12.9,1.4,7.1,7.4,0.0,...,0.345,0.319,0.320,0.327,0.019,6.9,7.0,7.7,8.9,10.9
307,64758,20231104,2.622,-76.25,42.44,13.4,5.7,9.6,9.5,0.0,...,0.338,0.313,0.312,0.315,0.024,8.1,7.9,8.3,8.9,10.6


In [5]:
Date = pd.to_datetime(df.LST_DATE, format='%Y%m%d', errors='coerce')
df['Time'] = Date

In [6]:
df.columns

Index(['WBANNO', 'LST_DATE', 'CRX_VN', 'LONGITUDE', 'LATITUDE', 'T_DAILY_MAX',
       'T_DAILY_MIN', 'T_DAILY_MEAN', 'T_DAILY_AVG', 'P_DAILY_CALC',
       'SOLARAD_DAILY', 'SUR_TEMP_DAILY_TYPE', 'SUR_TEMP_DAILY_MAX',
       'SUR_TEMP_DAILY_MIN', 'SUR_TEMP_DAILY_AVG', 'RH_DAILY_MAX',
       'RH_DAILY_MIN', 'RH_DAILY_AVG', 'SOIL_MOISTURE_5_DAILY',
       'SOIL_MOISTURE_10_DAILY', 'SOIL_MOISTURE_20_DAILY',
       'SOIL_MOISTURE_50_DAILY', 'SOIL_MOISTURE_100_DAILY',
       'SOIL_TEMP_5_DAILY', 'SOIL_TEMP_10_DAILY', 'SOIL_TEMP_20_DAILY',
       'SOIL_TEMP_50_DAILY', 'SOIL_TEMP_100_DAILY', 'Time'],
      dtype='object')

In [7]:
data = df[['T_DAILY_MAX',
       'T_DAILY_MIN', 'T_DAILY_MEAN', 'T_DAILY_AVG', 'P_DAILY_CALC',
       'SOLARAD_DAILY', 'SUR_TEMP_DAILY_MAX',
       'SUR_TEMP_DAILY_MIN', 'SUR_TEMP_DAILY_AVG', 'RH_DAILY_MAX',
       'RH_DAILY_MIN', 'RH_DAILY_AVG']]

In [8]:
data.index = df['Time']

In [9]:
# check for N/A
data.min()

T_DAILY_MAX          -17.90
T_DAILY_MIN          -30.90
T_DAILY_MEAN         -24.40
T_DAILY_AVG          -23.20
P_DAILY_CALC           0.00
SOLARAD_DAILY          0.06
SUR_TEMP_DAILY_MAX   -16.50
SUR_TEMP_DAILY_MIN   -36.00
SUR_TEMP_DAILY_AVG   -23.60
RH_DAILY_MAX          51.50
RH_DAILY_MIN          11.90
RH_DAILY_AVG          36.40
dtype: float64

In [10]:
data.isna().sum()

T_DAILY_MAX             76
T_DAILY_MIN             76
T_DAILY_MEAN            76
T_DAILY_AVG             77
P_DAILY_CALC            47
SOLARAD_DAILY           56
SUR_TEMP_DAILY_MAX     438
SUR_TEMP_DAILY_MIN     438
SUR_TEMP_DAILY_AVG      56
RH_DAILY_MAX          2405
RH_DAILY_MIN          2405
RH_DAILY_AVG          2405
dtype: int64

In [11]:
# Check data types
data.dtypes

T_DAILY_MAX           float64
T_DAILY_MIN           float64
T_DAILY_MEAN          float64
T_DAILY_AVG           float64
P_DAILY_CALC          float64
SOLARAD_DAILY         float64
SUR_TEMP_DAILY_MAX    float64
SUR_TEMP_DAILY_MIN    float64
SUR_TEMP_DAILY_AVG    float64
RH_DAILY_MAX          float64
RH_DAILY_MIN          float64
RH_DAILY_AVG          float64
dtype: object

In [12]:
data.shape

(6949, 12)

In [13]:
# forward fill the missing values
data.ffill(axis = 0, inplace = True)

In [14]:
data

,T_DAILY_MAX,T_DAILY_MIN,T_DAILY_MEAN,T_DAILY_AVG,P_DAILY_CALC,SOLARAD_DAILY,SUR_TEMP_DAILY_MAX,SUR_TEMP_DAILY_MIN,SUR_TEMP_DAILY_AVG,RH_DAILY_MAX,RH_DAILY_MIN,RH_DAILY_AVG
Time,,,,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-10-28,12.7,-0.3,6.2,5.0,0.0,12.67,NaN,NaN,6.5,NaN,NaN,NaN
2004-10-29,16.3,2.5,9.4,9.7,0.0,8.99,NaN,NaN,10.2,NaN,NaN,NaN
2004-10-30,17.5,10.5,14.0,14.5,1.8,4.14,NaN,NaN,13.5,NaN,NaN,NaN
2004-10-31,17.0,9.1,13.1,12.6,0.0,3.42,NaN,NaN,10.8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-01,3.5,-2.2,0.6,0.2,3.0,5.25,9.0,-4.6,0.3,89.9,69.2,83.8
2023-11-02,6.1,-3.3,1.4,1.0,0.0,6.21,12.1,-5.2,0.6,90.0,53.4,75.2
2023-11-03,12.9,1.4,7.1,7.4,0.0,10.68,13.5,-1.1,5.7,63.6,26.1,43.0


In [15]:
# drop NaN at the top
data.dropna(inplace = True)

In [16]:
# set target
data['target'] = data['T_DAILY_AVG']

In [17]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, shuffle = False)

In [18]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [19]:
train.shape

(3672, 13)

In [20]:
test.shape

(918, 13)

In [21]:
# splitting data into sequences
def split_sequences(features, target, seq_len, forecast_len):
    X,y = list(), list()
    for i in range(len(features)):
        end_input = i + seq_len
        end_predict = end_input + forecast_len
        if end_predict > len(features)-1:
            break
        seq_x, seq_y = features[i:end_input,:], target[end_input:end_predict]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float64), tf.convert_to_tensor(y, dtype=tf.float64)

# Define Model

In [22]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RNN, LSTMCell, Input, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

class MyModel(tf.keras.Model):

    def __init__(self, input_shape, output_shape, name = 'BiLSTM-FC'):
        super().__init__(name = name)
        self.input_layer = Input(shape = input_shape, name = 'input')
        self.lstm1 = Bidirectional(LSTM(units=30, activation = 'tanh', input_shape = input_shape, return_sequences=False, name = 'lstm_1'),name = 'bilstm_1')
        self.dense1 = Dense(units=output_shape, activation = 'relu', name = 'dense_1')
        #self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.lstm1(inputs)
        x = self.dense1(x)
        #if training:
        #  x = self.dropout(x, training=training)
        return x

    def summary(self):
        model = Model(inputs = [self.input_layer], outputs = self.call(self.input_layer), name = self.name)
        return model.summary()

# Model Training
## input length : output length = 16:4

In [23]:
# prepare sequences
seq_len = 16
forecast_len = 4
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [24]:
X_train.shape

TensorShape([3652, 16, 12])

In [25]:
y_train.shape

TensorShape([3652, 4])

In [26]:
# create model instance
model_name = 'BiLSTM_16-4'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_16-4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 16, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 4)                 244       
                                                                 
Total params: 10,564
Trainable params: 10,564
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)
# traning for > 40 epoch starts to overfit
# save trained model
model.save('./LSTM/models_daily_v2/' + model_name)

Epoch 1/30
33/33 [==============================] - 3s 24ms/step - loss: 0.0350 - mse: 0.0350 - acc: 0.2474 - val_loss: 0.0089 - val_mse: 0.0089 - val_acc: 0.3142
Epoch 2/30
33/33 [==============================] - 0s 8ms/step - loss: 0.0089 - mse: 0.0089 - acc: 0.2663 - val_loss: 0.0067 - val_mse: 0.0067 - val_acc: 0.2705
Epoch 3/30
33/33 [==============================] - 0s 7ms/step - loss: 0.0080 - mse: 0.0080 - acc: 0.2833 - val_loss: 0.0068 - val_mse: 0.0068 - val_acc: 0.3087
Epoch 4/30
33/33 [==============================] - 0s 7ms/step - loss: 0.0078 - mse: 0.0078 - acc: 0.2995 - val_loss: 0.0069 - val_mse: 0.0069 - val_acc: 0.2951
Epoch 5/30
33/33 [==============================] - 0s 8ms/step - loss: 0.0077 - mse: 0.0077 - acc: 0.3110 - val_loss: 0.0067 - val_mse: 0.0067 - val_acc: 0.3333
Epoch 6/30
33/33 [==============================] - 0s 7ms/step - loss: 0.0076 - mse: 0.0076 - acc: 0.3390 - val_loss: 0.0065 - val_mse: 0.0065 - val_acc: 0.2923
Epoch 7/30
33/33 [=========

INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_16-4\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_16-4\assets


In [29]:
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

29/29 [==============================] - 0s 1ms/step


In [30]:
from sklearn.metrics import mean_squared_error
print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.006660300351272238
test set: 0.006566365723436761


## input length : output length = 24:6

In [31]:
# reset memory
tf.Graph().as_default()

# prepare sequences
seq_len = 24
forecast_len = 6
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [35]:
# create model instance
model_name = 'BiLSTM_24-6'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_24-6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 24, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 6)                 366       
                                                                 
Total params: 10,686
Trainable params: 10,686
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=50,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/50
33/33 [==============================] - 3s 29ms/step - loss: 0.0961 - mse: 0.0961 - acc: 0.1706 - val_loss: 0.0741 - val_mse: 0.0741 - val_acc: 0.1781
Epoch 2/50
33/33 [==============================] - 0s 9ms/step - loss: 0.0719 - mse: 0.0719 - acc: 0.1761 - val_loss: 0.0720 - val_mse: 0.0720 - val_acc: 0.1534
Epoch 3/50
33/33 [==============================] - 0s 9ms/step - loss: 0.0713 - mse: 0.0713 - acc: 0.1984 - val_loss: 0.0719 - val_mse: 0.0719 - val_acc: 0.1589
Epoch 4/50
33/33 [==============================] - 0s 9ms/step - loss: 0.0710 - mse: 0.0710 - acc: 0.2109 - val_loss: 0.0716 - val_mse: 0.0716 - val_acc: 0.2055
Epoch 5/50
33/33 [==============================] - 0s 9ms/step - loss: 0.0709 - mse: 0.0709 - acc: 0.2063 - val_loss: 0.0719 - val_mse: 0.0719 - val_acc: 0.2274
Epoch 6/50
33/33 [==============================] - 0s 10ms/step - loss: 0.0709 - mse: 0.0709 - acc: 0.2121 - val_loss: 0.0714 - val_mse: 0.0714 - val_acc: 0.1973
Epoch 7/50
33/33 [========

In [37]:
# evaluate the model
# save trained model
model.save('./LSTM/models_daily_v2/' + model_name)
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_24-6\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_24-6\assets


28/28 [==============================] - 0s 2ms/step
mean_squared_error
train set: 0.007152441633439636
test set: 0.006911209424476951


## input length : output length = 32:8

In [38]:
# reset memory
tf.Graph().as_default()

# prepare sequences
seq_len = 32
forecast_len = 8
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [43]:
# create model instance
model_name = 'BiLSTM_32-8'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_32-8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 8)                 488       
                                                                 
Total params: 10,808
Trainable params: 10,808
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=50,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/50
33/33 [==============================] - 3s 34ms/step - loss: 0.0778 - mse: 0.0778 - acc: 0.1258 - val_loss: 0.0582 - val_mse: 0.0582 - val_acc: 0.1099
Epoch 2/50
33/33 [==============================] - 1s 17ms/step - loss: 0.0569 - mse: 0.0569 - acc: 0.1340 - val_loss: 0.0562 - val_mse: 0.0562 - val_acc: 0.1566
Epoch 3/50
33/33 [==============================] - 1s 16ms/step - loss: 0.0558 - mse: 0.0558 - acc: 0.1405 - val_loss: 0.0558 - val_mse: 0.0558 - val_acc: 0.1429
Epoch 4/50
33/33 [==============================] - 1s 18ms/step - loss: 0.0560 - mse: 0.0560 - acc: 0.1509 - val_loss: 0.0559 - val_mse: 0.0559 - val_acc: 0.1621
Epoch 5/50
33/33 [==============================] - 1s 18ms/step - loss: 0.0557 - mse: 0.0557 - acc: 0.1671 - val_loss: 0.0557 - val_mse: 0.0557 - val_acc: 0.1703
Epoch 6/50
33/33 [==============================] - 1s 17ms/step - loss: 0.0554 - mse: 0.0554 - acc: 0.1784 - val_loss: 0.0560 - val_mse: 0.0560 - val_acc: 0.1291
Epoch 7/50
33/33 [====

In [45]:
# save trained model
model.save('./LSTM/models_daily_v2/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_32-8\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_32-8\assets


28/28 [==============================] - 0s 2ms/step
mean_squared_error
train set: 0.00758172590045909
test set: 0.0070981972944135965


## input length : output length = 40:10

In [46]:
# reset memory
tf.Graph().as_default()

# prepare sequences
seq_len = 40
forecast_len = 10
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [51]:
# create model instance
model_name = 'BiLSTM_40-10'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_40-10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 40, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 10)                610       
                                                                 
Total params: 10,930
Trainable params: 10,930
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
33/33 [==============================] - 4s 37ms/step - loss: 0.0497 - mse: 0.0497 - acc: 0.0979 - val_loss: 0.0086 - val_mse: 0.0086 - val_acc: 0.1570
Epoch 2/30
33/33 [==============================] - 1s 19ms/step - loss: 0.0094 - mse: 0.0094 - acc: 0.1120 - val_loss: 0.0082 - val_mse: 0.0082 - val_acc: 0.1433
Epoch 3/30
33/33 [==============================] - 1s 20ms/step - loss: 0.0088 - mse: 0.0088 - acc: 0.1237 - val_loss: 0.0080 - val_mse: 0.0080 - val_acc: 0.1515
Epoch 4/30
33/33 [==============================] - 1s 20ms/step - loss: 0.0087 - mse: 0.0087 - acc: 0.1350 - val_loss: 0.0082 - val_mse: 0.0082 - val_acc: 0.1102
Epoch 5/30
33/33 [==============================] - 1s 20ms/step - loss: 0.0085 - mse: 0.0085 - acc: 0.1402 - val_loss: 0.0076 - val_mse: 0.0076 - val_acc: 0.1708
Epoch 6/30
33/33 [==============================] - 1s 20ms/step - loss: 0.0087 - mse: 0.0087 - acc: 0.1448 - val_loss: 0.0077 - val_mse: 0.0077 - val_acc: 0.1653
Epoch 7/30
33/33 [====

In [53]:
# save trained model
model.save('./LSTM/models_daily_v2/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_40-10\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_40-10\assets


28/28 [==============================] - 0s 2ms/step
mean_squared_error
train set: 0.007782176415058797
test set: 0.007640175914499972
